<a href="https://colab.research.google.com/github/sahasswamy/frontend-project/blob/main/PolynomialRegression2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
df = pd.read_excel(r'/content/PhD_Final_TS_Analysis.xlsx')

In [47]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

In [48]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 7 )

In [49]:
# Feature Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

##Normally Polynomial regression does not have any specific hyperparameters, but 'poly_features__degree': [1, 2, 3, 4],  # Polynomial degrees to test
##  'ridge__alpha': [0.1, 1, 10, 100]       # Regularization strengths

In [50]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
# Define pipeline
pipeline = Pipeline([
    ('poly_features', PolynomialFeatures()),
    ('ridge', Ridge())
])
# Define parameter grid
param_grid = {
    'poly_features__degree': [1, 2, 3, 4],  # Polynomial degrees to test
    'ridge__alpha': [0.1, 1, 10, 100]      # Regularization strengths
}
# Grid search
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error', verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Best parameters
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", -grid_search.best_score_)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
Best Parameters: {'poly_features__degree': 2, 'ridge__alpha': 100}
Best Score: 6.3212130895414065


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ridge.py:215: LinAlgWarning: Ill-conditioned matrix (rcond=8.83544e-18): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


## Model Development using Pipeline
### Get all the predicted and actual values for training and testing

In [51]:
degree = 2
model = make_pipeline(PolynomialFeatures(degree), LinearRegression())
# Fit the model on training data
model.fit(X_train_scaled, y_train)
# Predictions for both train and test sets
y_pred_train = model.predict(X_train_scaled)
y_pred_test = model.predict(X_test_scaled)
# Evaluation Metrics for test set
r2 = r2_score(y_test, y_pred_test)
mae = mean_absolute_error(y_test, y_pred_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
mape = np.mean(np.abs((y_test - y_pred_test) / y_test)) * 100
vaf = 1 - np.var(y_test - y_pred_test) / np.var(y_test)
d_index = 1 - (np.sum((y_test - y_pred_test)**2) / np.sum((np.abs(y_pred_test - y_test.mean()) + np.abs(y_test - y_test.mean()))**2))
# Display metrics
print(f"R²: {r2:.3f}")
print(f"MAE: {mae:.3f}")
print(f"MAPE: {mape:.3f}%")
print(f"RMSE: {rmse:.3f}")
print(f"VAF: {vaf:.3f}")
print(f"Willmott's Index of Agreement: {d_index:.3f}")
# Combine Actual vs Predicted values into a single DataFrame
result_train = pd.DataFrame({'Actual': y_train, 'Predicted': y_pred_train, 'Dataset': 'Train'})
result_test = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred_test, 'Dataset': 'Test'})
result_df = pd.concat([result_train, result_test], ignore_index=True)
# Save results to an Excel file
output_file = 'Predicted_Values_TS1.xlsx'
result_df.to_excel(output_file, index=False)
print(f"Combined Actual vs Predicted values saved to '{output_file}'")

R²: 0.834
MAE: 2.175
MAPE: 16.221%
RMSE: 2.776
VAF: 0.863
Willmott's Index of Agreement: 0.957
Combined Actual vs Predicted values saved to 'Predicted_Values_TS1.xlsx'


# Code to get polynomial regression equation

In [24]:
df.describe()

,GMW,Jb,U,D10N,D50,D90,Tensile_Strength
count,298.000000,298.000000,298.000000,298.000000,298.000000,298.000000,298.000000
mean,27403.849584,6.661189,0.470142,57.477779,224.893708,4547.411085,15.671745
std,3577.494027,1.102314,0.082128,15.617126,78.665395,704.093507,6.766893
min,15924.000000,3.911000,0.255000,35.672000,112.433000,3597.271000,5.200000
25%,25107.111952,5.852026,0.409333,43.216000,142.126250,3884.238000,9.165000
50%,27127.439375,6.642552,0.469947,53.660500,224.171000,4346.863500,15.075000
75%,29671.757817,7.452833,0.525347,72.111500,286.672250,5148.747000,20.287500
max,38225.000000,9.551000,0.676000,89.673000,372.467000,5939.277000,30.330000


In [ ]:
# Extract the coefficients and display the equation
# Create a pipeline with Polynomial Features and Linear Regression
poly = PolynomialFeatures(degree)
X_train_poly = poly.fit_transform(X_train)
model = LinearRegression()
model.fit(X_train_poly, y_train)
coefficients = model.coef_
intercept = model.intercept_
feature_names = poly.get_feature_names_out(['FI', 'GMQ', 'τ', 'D10', 'D50', 'D90'])

# Generate the equation
equation = f"{intercept:.4f}"
for coef, feature in zip(coefficients, feature_names):
    if coef != 0:
        equation += f" + ({coef:.4f})*{feature}"

print("\nPolynomial Regression Equation:")
print(equation)


Polynomial Regression Equation:
-2597.8872 + (-0.0037)*1 + (2.8082)*FI + (84.7164)*GMQ + (-17907.2701)*τ + (-321.3994)*D10 + (15.3414)*D50 + (4.0659)*D90 + (-0.0005)*FI^2 + (-0.0138)*FI GMQ + (-0.0769)*FI τ + (0.0303)*FI D10 + (-0.0054)*FI D50 + (0.0000)*FI D90 + (-0.2211)*GMQ^2 + (194.7063)*GMQ τ + (0.9216)*GMQ D10 + (-0.1336)*GMQ D50 + (-0.0106)*GMQ D90 + (-35639.8443)*τ^2 + (-218.7761)*τ D10 + (40.7139)*τ D50 + (0.6474)*τ D90 + (-0.7850)*D10^2 + (-0.0712)*D10 D50 + (0.0565)*D10 D90 + (0.0256)*D50^2 + (-0.0018)*D50 D90 + (-0.0006)*D90^2
